In [1]:
New Notebook

ERROR: Error in parse(text = x, srcfile = src): <text>:1:5: unexpected symbol
1: New Notebook
        ^


# Japanese Region Based on Height and Weight in Males

# Introduction

# Preliminary Exploratory Analysis

# Methods

# Expected Outcomes and Significance